### Behind the pipeline

In [3]:
import torch
import numpy as np
torch.set_printoptions(edgeitems=2, precision=2, linewidth=75)

让我们从一个完整的例子开始，看看当我们在第1章中执行以下代码时，在幕后发生了什么:

In [7]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
result = classifier([
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
])

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [8]:
result

[{'label': 'POSITIVE', 'score': 0.9598049521446228},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

让我们快速浏览一下这些。

### Preprocessing with a tokenizer

与其他神经网络一样，Transformer模型并不能直接处理原始文本，因此pipeline的第一步是将文本输入转换为模型可以理解的数字。为此，我们使用一个tokenizer，它将负责:
- 将输入拆分为称为标记的单词、子单词或符号(如标点符号)
- 将每个标记映射为一个整数
- 添加可能对模型有用的额外输入

所有这些预处理都需要以与模型预训练时完全相同的方式完成，因此我们首先需要从Model Hub下载该信息。为此，我们使用AutoTokenizer类及其from_pretrained()方法。使用模型的checkpoint名称，它将自动获取与模型的tokenizer相关的数据并缓存它(其仅在首次运行时下载)。

这里sentiment-analysis管线默认的checkpoint为 distilbert-base-uncased-finetuned-sst-2-english，因此可以运行如下代码：

In [9]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [10]:
tokenizer

DistilBertTokenizerFast(name_or_path='distilbert-base-uncased-finetuned-sst-2-english', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

一旦有了tokenizer，就可以直接将句子传递给它，这样就可以得到一个可以提供给模型的字典内容。剩下唯一要做的就是将输入id列表转换为tensor。
使用transformer时，不必担心使用哪个ML框架作为后端。它可能是PyTorch或TensorFlow，或某些模型的Flex。然而，Transformer模型只接受张量作为输入。如果这是你第一次听说张量，你可以把它们想象成NumPy数组。NumPy数组可以是标量(0D)，矢量(1D)，矩阵(2D)，或者具有更多维度。它实际上是一个张量。与其他ML框架的张量的行为类似，并且通常像NumPy数组一样简单地实例化。
指定我们想要返回的张量类型(PyTorch, TensorFlow或普通NumPy)，可以使用return_tensors参数:

In [12]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")

In [13]:
inputs

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662,
         12172,  2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,
             0,     0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}

这里先不用关心参数padding和truncation，稍后会做解释。这里只需记住：可以传递单句或列表，以及指定想要返回的tensor类型(如果没有传递类型，将得到一个列表的列表作为结果)。
输出本身是一个包含两个键input_ids和attention_mask的字典。input_ids包含两行整数(每个句子一行)，它们是每个句子中token的唯一标识符。我们将在本章后面解释什么是attention_mask。

### Going through the model

我们可以像下载tokenizer一样下载预训练模型。Transformers提供了一个AutoModel类，其也有from_pretrained()方法:

In [15]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

Some weights of the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing DistilBertModel: ['classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


在代码中，我们下载了之前在pipeline中使用的相同checkpoint(实际上它应该已经被缓存了)，并用它实例化了一个模型。该框架仅包含基本的Transformer模块: 给定一些输入，其输出hidden states(也称为features特征)的内容。对于每个模型输入，我们将检索一个高维向量，表示Transformer模型对该输入的上下文理解。
如果不明白这些，不要担心，我们稍后会再解释。
这些hidden states本身可能很有用，其通常是模型的另一部分输入，被称为head。在第一章中，不同的任务可以用相同的框架来执行，但是每个任务将有一个不同的头与之相关联。


#### A high-dimensional vector?